In [1]:
import numpy as np 
import pandas as pd
import random
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from torch.utils.data import random_split, DataLoader
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import torch.nn.functional as F
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Subset
from torchvision import models

import os
import time
from tqdm import tqdm
import threading
from concurrent.futures import ThreadPoolExecutor

/Users/efe/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ======================= PYTHON SETTINGS ======================= #
# =======================   GPU or CPU    ======================= #

device = torch.device("cpu")

# Check if GPU is available -> CUDA
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))

# Apple Silicon GPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
    out = torch.ones(1, device=device)
    print (out)
    print ("MPS device found. - Apple Silicon GPU")
else:
    print ("MPS device not found.")


print("Device:", device)

# =======================   Ranom Seeds   ======================= #
# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
# =============================================================== #

tensor([1.], device='mps:0')
MPS device found. - Apple Silicon GPU
Device: mps


In [4]:
# ======================= NORMALIZE PARAMS ======================= #
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to a consistent size
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(           # Normalize the images
        # average values of the red, green, 
        # and blue channels across all images in the ImageNet dataset.
        mean=[0.485, 0.456, 0.406], 
        # standard deviation of the red, green, and blue 
        # channels across all images in the ImageNet dataset.
        std=[0.229, 0.224, 0.225]   
    )
])

# ======================= DATA AUGMENTATION ======================= #
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),            # Resize images to a consistent size
    transforms.RandomHorizontalFlip(),        # Random horizontal flipping
    transforms.RandomRotation(15),            # Random rotation by 10 degrees
    transforms.ColorJitter(brightness=0.2,    # Adjust brightness, contrast, saturation, and hue
                           contrast=0.2,
                           saturation=0.2,
                           hue=0.1),
    transforms.ToTensor(),                    # Convert images to tensors
    transforms.Normalize(                     # Normalize the images
        mean=[0.485, 0.456, 0.406],           # Mean for ImageNet dataset
        std=[0.229, 0.224, 0.225]             # Standard deviation for ImageNet dataset
    )
])

In [5]:
# ======================= LOAD MAIN DATASET ======================= #
# Download and load the Food101 dataset
train_dataset = torchvision.datasets.Food101(
    root='./data',                 # Directory to save the downloaded data
    split='train',
    download=True,                 # Download the data if not present
    transform=train_transform            # Apply the train_transform to the data
)

test_dataset = torchvision.datasets.Food101(
    root='./data',                 # Directory to save the downloaded data
    split='test',
    download=True,                 # Download the data if not present
    transform=transform            # Apply the transform to the data
)

In [6]:
# def get_labels_from_loader(loader):
#     labels = []
#     for _, label in loader:
#         labels.extend(label.tolist())
#     return labels

# def get_subset_with_n_classes(original_dataset, n_classes, batch_size=64):
#     # Check if the dataset is a Subset
#     if isinstance(original_dataset, Subset):
#         # Create a DataLoader to extract labels from the Subset
#         subset_loader = DataLoader(original_dataset, batch_size=batch_size, shuffle=False)
#         # Extract labels
#         targets = get_labels_from_loader(subset_loader)
#     else:
#         # Try to access labels through the targets attribute
#         try:
#             targets = original_dataset.targets
#         except AttributeError:
#             # If targets attribute doesn't exist, use __getitem__ method
#             targets = [original_dataset[i][1] for i in range(len(original_dataset))]

#     # Find the unique classes and select the first n
#     unique_classes = torch.unique(torch.tensor(targets))
#     selected_classes = unique_classes[:n_classes].tolist()

#     # Get the indices of samples belonging to the selected classes
#     selected_indices = [i for i, t in enumerate(targets) if t in selected_classes]

#     # Handle if the dataset is already a Subset
#     if isinstance(original_dataset, Subset):
#         # Adjust indices to map to the original dataset
#         original_indices = [original_dataset.indices[i] for i in selected_indices]
#         # Create a new Subset from the original dataset
#         subset = Subset(original_dataset.dataset, original_indices)
#     else:
#         # Create a new Subset
#         subset = Subset(original_dataset, selected_indices)

#     return subset

def get_labels_from_loader(loader):
    # Concatenate all labels into a single NumPy array for efficiency
    return np.concatenate([labels.numpy() for _, labels in loader])

def get_subset_with_n_classes(original_dataset, n_classes, batch_size=64):
    # Check if the dataset is a Subset
    if isinstance(original_dataset, Subset):
        # Create a DataLoader with a larger batch size for efficiency
        subset_loader = DataLoader(original_dataset, batch_size=batch_size, shuffle=False)
        targets = get_labels_from_loader(subset_loader)
    else:
        # Try to access labels through the targets attribute
        # try:
        #     targets = np.array(original_dataset.targets)
        # except AttributeError:
            # Use list comprehension and NumPy for efficient label extraction
        targets = np.array([original_dataset[i][1] for i in range(len(original_dataset))])

    # Find the unique classes and select the first n
    unique_classes = np.unique(targets)
    selected_classes = unique_classes[:n_classes]

    # Vectorized operation for selecting indices
    selected_indices = np.where(np.isin(targets, selected_classes))[0]

    # Handle if the dataset is already a Subset
    if isinstance(original_dataset, Subset):
        original_indices = original_dataset.indices[selected_indices]
        subset = Subset(original_dataset.dataset, original_indices)
    else:
        subset = Subset(original_dataset, selected_indices)

    return subset



# Get a subset with the first n classes
n = 101  # For example, get the first 5 classes
# subset_train = get_subset_with_n_classes(train_dataset, n)
# subset_test =  get_subset_with_n_classes(test_dataset, n)

subset_train = train_dataset
subset_test = test_dataset

In [8]:
# test for small sets

print("subset_test", subset_train[0][0])
print("subset_test", subset_train[0][1])



subset_test tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.80

In [7]:
# =======================     DATA INFO    ======================= #
print('Train dataset length:',      len(train_dataset))
print('Test dataset length:',       len(test_dataset))
print('Classes:',                   train_dataset.classes)
print('Number of classes:',         len(train_dataset.classes))
print("===============================================")
print('Sample image shape:',        np.shape(train_dataset[0][0]))
# print('Sample image shape [0][0]:', train_dataset[0][0])
print('Sample image label [0][1]:', train_dataset[0][1])
print('Sample image class:',        train_dataset.classes[train_dataset[0][1]])

Train dataset length: 75750
Test dataset length: 25250
Classes: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobs

In [ ]:
# def display_image(dataset, index):
#     plt.imshow(dataset[index][0])                       # Plot the image
#     plt.title(train_dataset.classes[dataset[index][1]]) # Set the title 
# ======================== DISPLAY IMAGE (FROM MAIN DATASET) ======================== #
def display_image(dataset, index):
    image, label = dataset[index]
    image = to_pil_image(image)
    plt.imshow(image)
    plt.title(train_dataset.classes[label])

display_image(train_dataset, 33)

In [ ]:
# SHOW AN IMAGE FROM THE SUBSET

def display_image(dataset, index):
    image, label = dataset[index]
    image = to_pil_image(image)
    plt.imshow(image)
    plt.title(train_dataset.classes[label])

display_image(subset_train, 2620)


In [ ]:
# ================ SAMPLE IMAGE FOR ALL CLASSES ================ #
def display_each_class(dataset):
    plt.figure(figsize=(256, 256))  # Adjust the figure size as needed
    for idx in range(0, len(train_dataset.classes)):
        plt.subplot(26, 4, idx+1)
        plt.imshow(dataset[idx * 750][0])  # Indexing starts from 0
        plt.title(dataset.classes[dataset[idx * 750][1]])
        # plt.xticks([])
        # plt.yticks([])

    plt.tight_layout()  # Move this outside the loop
    # plt.show()
    plt.savefig("output.png")

display_each_class(train_dataset)

In [ ]:
# General function to display the histogram of the images
def list_image_paths(root_dir = "data/food-101/images", extensions=['.jpg', '.jpeg', '.png']):
    image_paths = []

    for root, dirs, files in os.walk(root_dir):
        #write in red
        # print('\033[91m' + root)
        for file in files:
            #write in blue
            # print('\033[94m' + file)
    
            if any(file.lower().endswith(ext) for ext in extensions):
                image_paths.append(os.path.join(root, file))
                break
            

    return image_paths

def display_histogram(image_paths):
    # Load images and extract pixel values
    images = [np.array(Image.open(image_path).convert("RGB")) for image_path in image_paths]
    
    # Flatten the 3D array of pixel values into a 1D array
    pixel_values_flat = np.concatenate([image.flatten() for image in images])

    # print('Total number of pixels:', len(pixel_values_flat))

    # Make a histogram for the pixel values
    plt.figure(figsize=(20, 10))
    plt.hist(pixel_values_flat, bins=20, range=(0, 255), color='blue', edgecolor='black')
    plt.title('Distribution of Pixel Intensities in the Dataset')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.show()

test = list_image_paths()
# print(len(test))
# print(test[0:101])
display_histogram(test)

In [9]:
# =======================   CNN MODEL    ======================= #
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Input shape: [batch_size, 3, 256, 256]
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # Fully connected layer
        self.fc1 = nn.Linear(64 * 64 * 64, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 64 * 64)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
# =======================   CNN MODEL    ======================= #
class SimpleCNN2(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN2, self).__init__()
        # Input shape: [batch_size, 3, 256, 256]
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # Fully connected layer
        self.fc1 = nn.Linear(128 * 128 * 128, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 128 * 128)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [11]:
class SimpleCNN3(nn.Module):
    def __init__(self, num_classes, dropout_prob=0.2):
        super(SimpleCNN3, self).__init__()
        # Input shape: [batch_size, 3, 256, 256]
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm after the first convolutional layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)  # BatchNorm after the second convolutional layer
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  # BatchNorm after the third convolutional layer
        # Fully connected layer
        self.fc1 = nn.Linear(128 * 128 * 128, 1024)
        self.bn_fc1 = nn.BatchNorm1d(1024)  # BatchNorm after the first fully connected layer
        self.dropout1 = nn.Dropout(dropout_prob)  # Dropout before the first fully connected layer
        self.fc2 = nn.Linear(1024, 1024)
        self.bn_fc2 = nn.BatchNorm1d(1024)  # BatchNorm after the second fully connected layer
        self.dropout2 = nn.Dropout(dropout_prob)  # Dropout before the second fully connected layer
        self.fc3 = nn.Linear(1024, 1024)
        self.bn_fc3 = nn.BatchNorm1d(1024)  # BatchNorm after the third fully connected layer
        self.dropout3 = nn.Dropout(dropout_prob)  # Dropout before the third fully connected layer
        self.fc4 = nn.Linear(1024, 1024)
        self.fc5 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Apply BatchNorm after the first convolutional layer
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Apply BatchNorm after the second convolutional layer
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Apply BatchNorm after the third convolutional layer
        x = x.view(-1, 128 * 128 * 128)  # Flatten the tensor
        x = F.relu(self.bn_fc1(self.fc1(x)))  # Apply BatchNorm after the first fully connected layer
        x = self.dropout1(x)  # Apply dropout before the first fully connected layer
        x = F.relu(self.bn_fc2(self.fc2(x)))  # Apply BatchNorm after the second fully connected layer
        x = self.dropout2(x)  # Apply dropout before the second fully connected layer
        x = F.relu(self.bn_fc3(self.fc3(x)))  # Apply BatchNorm after the third fully connected layer
        x = self.dropout3(x)  # Apply dropout before the third fully connected layer
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [12]:
class CustomImageClassifier(nn.Module):
   def __init__(self, num_classes):
       super(CustomImageClassifier, self).__init__()
      
       # Load a pre-trained model (e.g., ResNet)
       self.model = models.resnet18(pretrained=True)
      
       # Freeze the parameters of the model
       for param in self.model.parameters():
           param.requires_grad = False


       # Assuming ResNet18 is used, the in_features for the first added linear layer
       in_features = self.model.fc.in_features


       # Replace the fully connected layer
       self.model.fc = nn.Sequential(
           nn.Linear(in_features, 512),
           nn.ReLU(),
           nn.BatchNorm1d(512),
           nn.Dropout(0.5),
           nn.Linear(512, 256),
           nn.ReLU(),
           nn.BatchNorm1d(256),
           nn.Dropout(0.5),
           nn.Linear(256, num_classes)
       )


   def forward(self, x):
       return self.model(x)

In [ ]:
# ======================= SMALL MODEL TRAINING FOR 5 CLASSES ======================= #
# Hyperparameters
num_epochs = 20
batch_size = 32
num_classes = 5 # ? temporary 5
learning_rate = 0.001
num_folds = 5  # Number of folds for K-fold cross-validation

# Initialize KFold for cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

best_model = None
best_val_accuracy = 0.0

for fold, (train_indices, val_indices) in enumerate(kf.split(subset_train)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Initialize model, loss function, and optimizer
    model = CustomImageClassifier(num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Split the data into training and validation sets for this fold
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(subset_train, batch_size=batch_size, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(subset_train, batch_size=batch_size, sampler=val_sampler)

    for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch"):
        # Training loop
        model.train()
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch"):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        total_correct = 0
        total_samples = 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} (Validation)", unit="batch"):
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                total_correct += (predicted == labels).sum().item()

        val_accuracy = total_correct / total_samples
        
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation Accuracy: {val_accuracy:.4f}")

        # Check if this model has the best validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()

# After K-fold cross-validation, use the best model for testing
model.load_state_dict(best_model)
model.eval()

# Save the best model
torch.save(best_model, 'bestmodel.pt')

# Define the data loader for testing
test_loader = torch.utils.data.DataLoader(subset_test, batch_size=batch_size, shuffle=False)

# Testing loop
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

test_accuracy = total_correct / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

In [18]:
# ======================= BIG MODEL TRAINING FOR 101 CLASSES ======================= #

torch.backends.cudnn.enabled = False

print("Device:", device)
if device == torch.device("cpu"):
    print("Training on CPU - not recommended!")
    print("ABORTING")
    exit()

# Hyperparameters
num_epochs = 20
batch_size = 32
num_classes = 101
learning_rate = 0.001
num_folds = 5
seed = 42

# Initialize KFold for cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

best_model = None
best_val_accuracy = 0.0

for fold, (train_indices, val_indices) in enumerate(kf.split(subset_train)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Initialize model, loss function, and optimizer
    model = CustomImageClassifier(num_classes).to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Split the data into training and validation sets for this fold
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    train_loader = DataLoader(subset_train, batch_size=batch_size, sampler=train_sampler, pin_memory=True, num_workers=8)
    val_loader = DataLoader(subset_train, batch_size=batch_size, sampler=val_sampler, pin_memory=True, num_workers=8)

    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch"):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            with torch.cuda.amp.autocast("mps"):
                outputs = model(images.to(device))
                loss = criterion(outputs, labels.to(device))

            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        total_correct = 0
        total_samples = 0
        with torch.cuda.amp.autocast("mps"):
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                total_correct += (predicted == labels).sum().item()

        val_accuracy = total_correct / total_samples

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation Accuracy: {val_accuracy:.4f}")

        # Check if this model has the best validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()

# After K-fold cross-validation, use the best model for testing
model.load_state_dict(best_model)
model.eval()

# Save the best model
torch.save(best_model, 'bestmodel.pt')

# Define the data loader for testing
test_loader = DataLoader(subset_test, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=8)

# Testing loop
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels.to(device)).sum().item()

test_accuracy = total_correct / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

/Users/efe/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/efe/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Device: mps
Fold 1/5


Training:   0%|          | 0/20 [00:00<?, ?epoch/s]/Users/efe/miniconda3/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Training:   5%|▌         | 1/20 [02:45<52:25, 165.54s/epoch]

Epoch [1/20] - Validation Accuracy: 0.4523


Training:  10%|█         | 2/20 [05:30<49:38, 165.49s/epoch]

...

Epoch [18/20] - Validation Accuracy: 0.5190


Training:  95%|█████████▌| 19/20 [52:20<02:44, 164.75s/epoch]

Epoch [19/20] - Validation Accuracy: 0.5211


Training: 100%|██████████| 20/20 [55:03<00:00, 165.20s/epoch]

Epoch [20/20] - Validation Accuracy: 0.5232


In [7]:
# Nihat - 21 Ocak - 101 class 
class CustomImageClassifier2(nn.Module):
   def __init__(self, num_classes):
       super(CustomImageClassifier2, self).__init__()
      
       # Load a pre-trained model (e.g., ResNet)
       self.model = models.resnet50(pretrained=True)
      
       # Freeze the parameters of the model
       for param in self.model.parameters():
           param.requires_grad = False


       # Assuming ResNet18 is used, the in_features for the first added linear layer
       in_features = self.model.fc.in_features


       # Replace the fully connected layer
       self.model.fc = nn.Sequential(
           nn.Linear(in_features, 1024),
           nn.ReLU(),
           nn.BatchNorm1d(1024),
           nn.Dropout(0.5),
           nn.Linear(1024, 512),
           nn.ReLU(),
           nn.BatchNorm1d(512),
           nn.Dropout(0.5),
           nn.Linear(512, 256),
           nn.ReLU(),
           nn.BatchNorm1d(256),
           nn.Dropout(0.5),
           nn.Linear(256, num_classes)
       )


   def forward(self, x):
       return self.model(x)

In [19]:
# Nihat - 21 Ocak - 101 class 
# ======================= BIG MODEL TRAINING FOR 101 CLASSES ======================= #

torch.backends.cudnn.enabled = False

print("Device:", device)
if device == torch.device("cpu"):
    print("Training on CPU - not recommended!")
    print("ABORTING")
    exit()

# Hyperparameters
num_epochs = 20
batch_size = 256
num_classes = 101
learning_rate = 0.001
num_folds = 5
seed = 42

# Initialize KFold for cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

best_model = None
best_val_accuracy = 0.0

for fold, (train_indices, val_indices) in enumerate(kf.split(subset_train)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Initialize model, loss function, and optimizer
    model = CustomImageClassifier2(num_classes).to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Split the data into training and validation sets for this fold
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    train_loader = DataLoader(subset_train, batch_size=batch_size, sampler=train_sampler, pin_memory=True, num_workers=8)
    val_loader = DataLoader(subset_train, batch_size=batch_size, sampler=val_sampler, pin_memory=True, num_workers=8)

    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch"):
        model.train()

        running_loss = 0.0  # Initialize running loss for the epoch
        num_batches = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            with torch.cuda.amp.autocast("mps"):
                outputs = model(images.to(device))
                loss = criterion(outputs, labels.to(device))

            loss.backward()
            optimizer.step()

            running_loss += loss.item()  # Accumulate the loss
            num_batches += 1

        average_training_loss = running_loss / num_batches
        
        # Validation loop
        model.eval()
        total_correct = 0
        total_samples = 0

        val_running_loss = 0.0

        with torch.cuda.amp.autocast("mps"):
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()


                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                total_correct += (predicted == labels).sum().item()

        val_accuracy = total_correct / total_samples
        average_val_loss = val_running_loss / len(val_loader)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_training_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {average_val_loss:.4f}")

        # Check if this model has the best validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()

# After K-fold cross-validation, use the best model for testing
model.load_state_dict(best_model)
model.eval()

# Save the best model
torch.save(best_model, 'bestmodel.pt')

# Define the data loader for testing
test_loader = DataLoader(subset_test, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=8)

# Testing loop
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels.to(device)).sum().item()

test_accuracy = total_correct / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

Device: mps
Fold 1/5


/Users/nihat/mambaforge/envs/2023-06-05-neuralnetworks/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Training:   5%|▌         | 1/20 [11:22<3:36:05, 682.37s/epoch]

Epoch [1/20] - Training Loss: 3.4087, Validation Accuracy: 0.3710, Validation Loss: 2.5550


Training:  10%|█         | 2/20 [22:21<3:20:32, 668.49s/epoch]

Epoch [2/20] - Training Loss: 2.7423, Validation Accuracy: 0.4071, Validation Loss: 2.3727


...

Epoch [3/20] - Training Loss: 2.6183, Validation Accuracy: 0.4267, Validation Loss: 2.2819


Training:  20%|██        | 4/20 [43:17<2:53:38, 651.17s/epoch]

Epoch [4/20] - Training Loss: 2.5398, Validation Accuracy: 0.4413, Validation Loss: 2.2256


ERROR: Unexpected segmentation fault encountered in worker.
Training:  20%|██        | 4/20 [53:30<3:34:00, 802.54s/epoch]


RuntimeError: DataLoader worker (pid 35050) is killed by signal: Segmentation fault: 11. 

In [8]:
# Efe - 21 Ocak - 101 class  -> assigning every Fold to a thread
# ======================= BIG MODEL TRAINING FOR 101 CLASSES ======================= #

# torch.backends.cudnn.enabled = False

# print("Device:", device)
# if device == torch.device("cpu"):
#     print("Training on CPU - not recommended!")
#     print("ABORTING")
#     exit()

# # Hyperparameters
# num_epochs = 2      # ! Make it > 20 when you are ready to train the model for real
# batch_size = 32
# num_classes = 101
# learning_rate = 0.001
# num_folds = 5
# seed = 42

# # Initialize KFold for cross-validation
# kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

# best_model = None
# best_val_accuracy = 0.0

# for fold, (train_indices, val_indices) in enumerate(kf.split(subset_train)):
#     print(f"Fold {fold + 1}/{num_folds}")

#     # Initialize model, loss function, and optimizer
#     model = CustomImageClassifier2(num_classes).to(device)
#     criterion = nn.CrossEntropyLoss().to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#     # Split the data into training and validation sets for this fold
#     train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
#     val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

#     train_loader = DataLoader(subset_train, batch_size=batch_size, sampler=train_sampler, pin_memory=True, num_workers=8)
#     val_loader = DataLoader(subset_train, batch_size=batch_size, sampler=val_sampler, pin_memory=True, num_workers=8)

#     # Training loop
#     for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch"):
#         model.train()

#         running_loss = 0.0  # Initialize running loss for the epoch
#         num_batches = 0

#         for images, labels in train_loader:
#             optimizer.zero_grad()
#             with torch.cuda.amp.autocast("mps"):
#                 outputs = model(images.to(device))
#                 loss = criterion(outputs, labels.to(device))

#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()  # Accumulate the loss
#             num_batches += 1

#         average_training_loss = running_loss / num_batches
        
#         # Validation loop
#         model.eval()
#         total_correct = 0
#         total_samples = 0

#         val_running_loss = 0.0

#         with torch.cuda.amp.autocast("mps"):
#             for images, labels in val_loader:
#                 images = images.to(device)
#                 labels = labels.to(device)

#                 outputs = model(images)
                
#                 loss = criterion(outputs, labels)
#                 val_running_loss += loss.item()


#                 _, predicted = torch.max(outputs, 1)
#                 total_samples += labels.size(0)
#                 total_correct += (predicted == labels).sum().item()

#         val_accuracy = total_correct / total_samples
#         average_val_loss = val_running_loss / len(val_loader)

#         print(f"Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_training_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {average_val_loss:.4f}")

#         # Check if this model has the best validation accuracy
#         if val_accuracy > best_val_accuracy:
#             best_val_accuracy = val_accuracy
#             best_model = model.state_dict()

# # After K-fold cross-validation, use the best model for testing
# model.load_state_dict(best_model)
# model.eval()

# # Save the best model
# torch.save(best_model, 'bestmodel.pt')

# # Define the data loader for testing
# test_loader = DataLoader(subset_test, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=8)

# # Testing loop
# model.eval()
# total_correct = 0
# total_samples = 0
# with torch.no_grad():
#     for images, labels in test_loader:
#         outputs = model(images.to(device))
#         _, predicted = torch.max(outputs, 1)
#         total_samples += labels.size(0)
#         total_correct += (predicted == labels.to(device)).sum().item()

# test_accuracy = total_correct / total_samples
# print(f"Test Accuracy: {test_accuracy:.4f}")





# Function to train a model for a fold
def train_fold(fold, train_indices, val_indices):
    global best_model
    global best_val_accuracy

    print(f"Fold {fold + 1}/{num_folds}")

    # Initialize model, loss function, and optimizer
    model = CustomImageClassifier2(num_classes).to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Split the data into training and validation sets for this fold
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    train_loader = DataLoader(subset_train, batch_size=batch_size, sampler=train_sampler, pin_memory=True, num_workers=4)
    val_loader = DataLoader(subset_train, batch_size=batch_size, sampler=val_sampler, pin_memory=True, num_workers=4)

    # Training loop
    for epoch in range(num_epochs):

        # Print the worker ID
        print(f"Worker {threading.get_ident()} - Epoch {epoch + 1}/{num_epochs}")

        model.train()

        running_loss = 0.0  # Initialize running loss for the epoch
        num_batches = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            with torch.cuda.amp.autocast("mps"):
                outputs = model(images.to(device))
                loss = criterion(outputs, labels.to(device))

            loss.backward()
            optimizer.step()

            running_loss += loss.item()  # Accumulate the loss
            num_batches += 1

        average_training_loss = running_loss / num_batches

        # Validation loop
        model.eval()
        total_correct = 0
        total_samples = 0

        val_running_loss = 0.0

        with torch.cuda.amp.autocast("mps"):
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                total_correct += (predicted == labels).sum().item()

        val_accuracy = total_correct / total_samples
        average_val_loss = val_running_loss / len(val_loader)

        print(f"Fold {fold + 1} - Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_training_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {average_val_loss:.4f}")

        # Check if this model has the best validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()
        
        # Print the worker ID - finished
        print(f"Worker {threading.get_ident()} - Epoch {epoch + 1}/{num_epochs} finished")

# Set other hyperparameters and initialize KFold
num_epochs = 1
batch_size = 256
num_classes = 101
learning_rate = 0.001
num_folds = 2
seed = 42
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

# Store best model and accuracy globally
best_model = None
best_val_accuracy = 0.0

# Create threads for each fold using ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(train_fold, fold, train_indices, val_indices) for fold, (train_indices, val_indices) in enumerate(kf.split(subset_train))]


# Wait for all threads to finish
for future in futures:
    print("Waiting for thread to finish...")
    future.result()
    print("Threads finished!")

# After K-fold cross-validation, use the best model for testing
model = CustomImageClassifier2(num_classes).to(device)
model.load_state_dict(best_model)
model.eval()

# Save the best model
torch.save(best_model, 'bestmodel.pt')

# Define the data loader for testing
test_loader = DataLoader(subset_test, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=8)

# Testing loop
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels.to(device)).sum().item()

test_accuracy = total_correct / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")


Fold 1/2
Fold 2/2


/Users/efe/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/efe/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Worker 12809433088 - Epoch 1/1Worker 12826259456 - Epoch 1/1



/Users/efe/miniconda3/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


KeyboardInterrupt: 

: 